Javier Ortiz de Artiñano. Trabajo de Fin de Grado 2023.

This script reads a bunch of downloaded genome assemblies (found in /data/assemblies) and a fasta file with $hcp$, $vgrG$ and $tssB$ locus names. Then, it takes the neighbor CDS loci of the specified loci from the genomes and writes their name. This script should be run after 'DBGenerator.bash'. It also requires a HcpVgrGTssB_loci.fasta file (found in /data).

In [1]:
# Packages
import sys, os
import re
from Bio import SeqIO,Seq

In [7]:
# Takes as input a genomic .gbff file and parses it. Generates a list with all CDS locus names and a dictionary with information of all loci.
# The dictionary has as keys the locus names, and as values a list with the following elements:
# 0: strand; 1: product (empty if None); 2: start; 3:end; 4: gene name (empty if None); 5: contig name
def make_list_and_dict(input_file):
    list_of_genes = []
    dict_of_genes = {}
    
    with open(input_file,"r") as input_handle:
        for record in SeqIO.parse(input_handle,"genbank"):
            for feature in record.features:
                if feature.type == 'CDS':
                    locus_tag = feature.qualifiers['locus_tag'][0]
                    list_of_genes.append(locus_tag)
                    start = feature.location.start
                    end = feature.location.end
                    strand = feature.location.strand
                    contig_name = record.id
                    try:
                        gene = feature.qualifiers['gene'][0]
                    except:
                        gene = "NA"
                    try:
                        product = feature.qualifiers['product'][0]
                    except:
                        product = "NA"
                    dict_of_genes[locus_tag] = [strand,product,start,end,gene,contig_name]
    return dict_of_genes, list_of_genes

# Returns the neighbourhood loci of a specified locus. n_context specifies 
# how many neighbor loci are taken at each side
def get_genomic_neighbourhood_list(locus, list_of_genes, n_context):
    if locus not in list_of_genes:
        print('Locus',locus, 'is not valid')
        return
    neighbourhood_list=[]
    pos_locus = list_of_genes.index(locus)
    for i in range(pos_locus-n_context,pos_locus+n_context+1):
        neighbourhood_list.append(list_of_genes[i])
    return neighbourhood_list

In [2]:
# SPECIFY HERE YOUR PROJECT DIRECTORY
project_directory = ''

Obtain a text file with the name and location of every genomic.gbff file.

In [105]:
regex = re.compile(r'.*genomic.gbff')
with open(project_directory + "/data/genbank_files.txt", "w") as fd:
    for root, dirs, files in os.walk(project_directory + "/data/assemblies"):
        for file in files:
            if regex.match(file):
                fd.write(f"{root}/{file}\n")

Obtain the locus names file, all_context_loci.txt. Format:

assembly_name!contig_id@locus_name

In [2]:
all_context_loci = []

all_hcp_vgrG_tssB_loci = SeqIO.parse(project_directory + "/data/HcpVgrGTssB_loci.fasta", "fasta")
list_of_assembly_files = []
with open(project_directory + "/data/all_context_loci.txt", "w") as fd, open(project_directory + "/data/genbank_files.txt", "r") as genbank_files:
    for assembly_file in genbank_files:
        list_of_assembly_files.append(assembly_file)
    for assembly_locus in all_hcp_vgrG_tssB_loci:
        assembly_name = re.sub(r'!.*$','',assembly_locus.id)
        for assembly_file in list_of_assembly_files:
            if assembly_name not in assembly_file:
                continue
            locus = re.sub(r'^.*@','',assembly_locus.id)
            dict_of_genes, list_of_genes = make_list_and_dict(assembly_file[:-1])
            neighbours_list = get_genomic_neighbourhood_list(locus, list_of_genes, 25) #25 neighbors at each side
            for neighbour in neighbours_list:
                if dict_of_genes[neighbour][5] != dict_of_genes[locus][5]: #checks that they are in the same contig
                    continue
                fd.write(f"{assembly_name}!{dict_of_genes[neighbour][5]}@{neighbour}\n")
            break

NameError: name 'SeqIO' is not defined

From all_context_loci.txt, obtaining the sequences from the first multifasta file is trivial (just convert both files to tsv format, sort them and join them in bash. Then, use the following command to change the format to fasta and finally get DB2).

In [14]:
SeqIO.convert(project_directory + "/data/brady_T6SS_context.tsv", "tab", project_directory + "/data/DB2.fasta", "fasta")

16601